In [3]:
import os
import json

from pathlib import Path
from collections import defaultdict, Counter
from itertools import chain, starmap

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
os.listdir("../loaders/mesowest/")

['.DS_Store',
 'mesowest_token.py',
 'janpsps.csv',
 '__pycache__',
 'janpsps.pickle',
 'mesowest_request.py',
 'sce_202101150000_202101180000.csv']

In [5]:
ws_mw = pd.read_csv("../loaders/mesowest/janpsps.csv")

In [7]:
ws_mw.columns = ws_mw.columns.str.title()
ws_mw["Station"] = ws_mw.Stid.str.replace("SE00", "")
# ws_mw.drop("Unnamed: 0", axis =1, inplace=True)
ws_mw.head()

,Status,Mnet_Id,Period_Of_Record,Elevation,Name,Stid,Sensor_Variables,Elev_Dem,Longitude,State,Observations,Restricted,Qc_Flagged,Latitude,Timezone,Id,Station
0,ACTIVE,231,"{'start': '2018-06-11T20:11:00Z', 'end': '2021...",6388,SCE Marion Ridge,SE001,"{'wind_speed': {'wind_speed_set_1': {}}, 'date...",6400.9,-116.73432,CA,"{'date_time': ['2021-01-18T00:00:00Z', '2021-0...",False,False,33.76594,America/Los_Angeles,70844,1
1,ACTIVE,231,"{'start': '2018-06-11T20:11:00Z', 'end': '2021...",5598,SCE Idyllwild,SE002,"{'wind_speed': {'wind_speed_set_1': {}}, 'date...",5587.3,-116.71663,CA,"{'date_time': ['2021-01-18T00:00:00Z', '2021-0...",False,False,33.75136,America/Los_Angeles,70845,2
2,ACTIVE,231,"{'start': '2018-06-11T20:11:00Z', 'end': '2021...",2750,SCE Saddle Peak,SE003,"{'wind_speed': {'wind_speed_set_1': {}}, 'date...",2732.9,-118.65901,CA,"{'date_time': ['2021-01-18T00:00:00Z', '2021-0...",False,False,34.07684,America/Los_Angeles,70846,3
3,ACTIVE,231,"{'start': '2018-06-14T20:32:00Z', 'end': '2021...",760,SCE Santa Anita Canyon 2,SE004,"{'wind_speed': {'wind_speed_set_1': {}}, 'date...",761.2,-118.02279,CA,"{'date_time': ['2021-01-18T00:00:00Z', '2021-0...",False,False,34.16891,America/Los_Angeles,70847,4
4,ACTIVE,231,"{'start': '2018-06-11T20:11:00Z', 'end': '2021...",1658,SCE Boquet Canyon,SE005,"{'wind_speed': {'wind_speed_set_1': {}}, 'date...",1663.4,-118.45637,CA,"{'date_time': ['2021-01-18T00:00:00Z', '2021-0...",False,False,34.49828,America/Los_Angeles,70848,5


In [224]:
# ws_mw.to_csv("../loaders/mesowest/janpsps.csv", index=False)

In [9]:
ws_mw.Observations[0]

"{'date_time': ['2021-01-18T00:00:00Z', '2021-01-18T00:10:00Z', '2021-01-18T00:20:00Z', '2021-01-18T00:30:00Z', '2021-01-18T00:40:00Z', '2021-01-18T00:50:00Z', '2021-01-18T01:00:00Z', '2021-01-18T01:10:00Z', '2021-01-18T01:20:00Z', '2021-01-18T01:30:00Z', '2021-01-18T01:40:00Z', '2021-01-18T01:50:00Z', '2021-01-18T02:00:00Z', '2021-01-18T02:10:00Z', '2021-01-18T02:20:00Z', '2021-01-18T02:30:00Z', '2021-01-18T02:40:00Z', '2021-01-18T02:50:00Z', '2021-01-18T03:00:00Z', '2021-01-18T03:10:00Z', '2021-01-18T03:20:00Z', '2021-01-18T03:30:00Z', '2021-01-18T03:40:00Z', '2021-01-18T03:50:00Z', '2021-01-18T04:00:00Z', '2021-01-18T04:10:00Z', '2021-01-18T04:20:00Z', '2021-01-18T04:30:00Z', '2021-01-18T04:40:00Z', '2021-01-18T04:50:00Z', '2021-01-18T05:00:00Z', '2021-01-18T05:10:00Z', '2021-01-18T05:20:00Z', '2021-01-18T05:30:00Z', '2021-01-18T05:40:00Z', '2021-01-18T05:50:00Z', '2021-01-18T06:00:00Z', '2021-01-18T06:10:00Z', '2021-01-18T06:20:00Z', '2021-01-18T06:30:00Z', '2021-01-18T06:40:00Z', 